In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from sklearn.metrics import classification_report,accuracy_score,roc_auc_score,confusion_matrix

In [25]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()  # Free GPU memory

In [3]:
# Data transformations
train_transforms = transforms.Compose([
    transforms.RandomRotation(15),  # 15-degree tilt
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((128, 128)),  # Reduce image size to save memory
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  # Reduce image size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load datasets
root = '/kaggle/input/tomato'
full_train_dataset = datasets.ImageFolder(os.path.join(root, 'train'), transform=train_transforms)
full_validation_dataset = datasets.ImageFolder(os.path.join(root, 'valid'), transform=test_transforms)

In [5]:
# Get class indices
class_names = full_train_dataset.classes
late_blight_index = class_names.index('Late_blight')
healthy_index = class_names.index('healthy')

def filter_late_blight_and_healthy(dataset):
    return [i for i, (_, label) in enumerate(dataset) if label in [late_blight_index, healthy_index]]

train_indices = filter_late_blight_and_healthy(full_train_dataset)
valid_indices = filter_late_blight_and_healthy(full_validation_dataset)

train_dataset = Subset(full_train_dataset, train_indices)
validation_dataset = Subset(full_validation_dataset, valid_indices)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)


In [6]:
# Define a smaller and memory-efficient CNN model
class TinyResNet(nn.Module):
    def __init__(self):
        super(TinyResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 1)  # Single output for binary classification
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.pool(F.relu_(self.bn1(self.conv1(x))))  # Use in-place ReLU
        x = self.pool(F.relu_(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = F.relu_(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Remove Sigmoid, use raw logits
        return x

cnn = TinyResNet().to(device)


In [7]:

# Training setup
num_epochs = 20
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for stability
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
scaler = torch.cuda.amp.GradScaler()  # Enable mixed precision training


<ipython-input-7-bf708d3040b5>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Enable mixed precision training


In [8]:
# Training loop
for epoch in range(num_epochs):
    cnn.train()
    total_loss = 0
    for img, label in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        img, label = img.to(device), (label == late_blight_index).float().to(device)  # 1 if Late Blight, else 0
        optimizer.zero_grad()
        
        with torch.amp.autocast(device_type='cuda'):  # Corrected mixed precision training
            y_pred = cnn(img).squeeze()
            loss = criterion(y_pred, label)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    
    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Avg Loss: {total_loss / len(train_loader):.4f}")


Epoch 1/20: 100%|██████████| 97/97 [00:47<00:00,  2.03it/s]


Epoch 1/20, Avg Loss: 0.6289


Epoch 2/20: 100%|██████████| 97/97 [00:47<00:00,  2.06it/s]


Epoch 2/20, Avg Loss: 0.2794


Epoch 3/20: 100%|██████████| 97/97 [00:46<00:00,  2.09it/s]


...

Epoch 18/20, Avg Loss: 0.1235


Epoch 19/20: 100%|██████████| 97/97 [00:46<00:00,  2.10it/s]


Epoch 19/20, Avg Loss: 0.1176


Epoch 20/20: 100%|██████████| 97/97 [00:46<00:00,  2.11it/s]

Epoch 20/20, Avg Loss: 0.1132


In [35]:
# Evaluation
from sklearn.metrics import classification_report
cnn.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(device), (label == late_blight_index).float().to(device)
        pred = cnn(img).squeeze()
        pred = (torch.sigmoid(pred) > 0.5).float().cpu().numpy()
        all_preds.extend(pred)
        all_labels.extend(label.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=['Healthy', 'Late Blight'], labels=[0, 1]))


              precision    recall  f1-score   support

     Healthy       0.98      0.97      0.97       805
 Late Blight       0.97      0.98      0.97       792

    accuracy                           0.97      1597
   macro avg       0.97      0.97      0.97      1597
weighted avg       0.97      0.97      0.97      1597



In [36]:
print(accuracy_score(all_labels, all_preds))

0.9730745147150908


In [37]:
print(confusion_matrix(all_labels, all_preds, labels=[0, 1]))

[[778  27]
 [ 16 776]]


In [42]:
print(roc_auc_score(all_labels, all_preds, labels=[0, 1]))

0.9685770750988142


In [19]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=8,stride=1,kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,stride=1,kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32,stride=1,kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=32,out_channels=64,stride=1,kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=64,out_channels=128,stride=1,kernel_size=3)
        self.fc1 = nn.Linear(128*4*4,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,32)
        self.out = nn.Linear(32,1)
        self.dropout = nn.Dropout(p=0.4)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv5(x))
        x = x.view(-1,128*4*4)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.out(x)
        return x

vgg_cnn = CNNModel().to(device)

In [20]:
num_epochs = 20
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for stability
optimizer = torch.optim.Adam(vgg_cnn.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
scaler = torch.cuda.amp.GradScaler()  # Enable mixed precision training

<ipython-input-20-58146ab3579b>:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Enable mixed precision training


In [21]:
# Training loop
for epoch in range(num_epochs):
    vgg_cnn.train()
    total_loss = 0
    for img, label in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        img, label = img.to(device), (label == late_blight_index).float().to(device)  # 1 if Late Blight, else 0
        optimizer.zero_grad()
        
        with torch.amp.autocast(device_type='cuda'):  # Corrected mixed precision training
            y_pred = vgg_cnn(img).squeeze()
            loss = criterion(y_pred, label)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    
    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Avg Loss: {total_loss / len(train_loader):.4f}")


Epoch 1/20: 100%|██████████| 97/97 [00:48<00:00,  2.01it/s]


Epoch 1/20, Avg Loss: 0.6241


Epoch 2/20: 100%|██████████| 97/97 [00:49<00:00,  1.96it/s]


Epoch 2/20, Avg Loss: 0.4453


Epoch 3/20: 100%|██████████| 97/97 [00:49<00:00,  1.96it/s]


...

Epoch 18/20, Avg Loss: 0.1083


Epoch 19/20: 100%|██████████| 97/97 [00:45<00:00,  2.12it/s]


Epoch 19/20, Avg Loss: 0.1133


Epoch 20/20: 100%|██████████| 97/97 [00:46<00:00,  2.07it/s]

Epoch 20/20, Avg Loss: 0.1029


In [38]:
# Evaluation
from sklearn.metrics import classification_report
vgg_cnn.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to(device), (label == late_blight_index).float().to(device)
        pred = vgg_cnn(img).squeeze()
        pred = (torch.sigmoid(pred) > 0.5).float().cpu().numpy()
        all_preds.extend(pred)
        all_labels.extend(label.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=['Healthy', 'Late Blight'], labels=[0, 1]))

              precision    recall  f1-score   support

     Healthy       0.96      0.98      0.97       805
 Late Blight       0.98      0.95      0.97       792

    accuracy                           0.97      1597
   macro avg       0.97      0.97      0.97      1597
weighted avg       0.97      0.97      0.97      1597



In [39]:
print(accuracy_score(all_labels, all_preds))

0.9686912961803381


In [40]:
print(roc_auc_score(all_labels, all_preds, labels=[0, 1]))

0.9685770750988142


In [41]:
print(confusion_matrix(all_labels, all_preds, labels=[0, 1]))

[[791  14]
 [ 36 756]]
